In [1]:
# DESeq2 analysis of CBTN RNA-seq

In [30]:
# Dependencies
suppressMessages(library(DESeq2))

# Data
path_to_counts =  "/Users/ochapman/Library/CloudStorage/OneDrive-SanfordBurnhamPrebysMedicalDiscoveryInstitute/projects/2023-pedpancan/data/gex/pbta-rsem-genes.expected_count.tsv"
path_to_annotation = "/Users/ochapman/Library/CloudStorage/OneDrive-SanfordBurnhamPrebysMedicalDiscoveryInstitute/projects/2023-pedpancan/data/gex/sample_phenotypes.csv"
load_inputs <- function(cts_path,annot_path){
    message("loading data...")
    cts = as.matrix(read.csv(cts_path,sep='\t',row.names="Gene_or_Transcript_ID",check.names=FALSE))
    annot = read.csv(annot_path,row.names=1)

    cts = round(cts) # counts must be integer
    cts <- cts[, rownames(annot)] # only include samples with metadata
    annot$age_at_diagnosis <- (annot$age_at_diagnosis - mean(annot$age_at_diagnosis)) / sd(annot$age_at_diagnosis)

    stopifnot(all(rownames(annot) == colnames(cts)))
    return(list(cts=cts,annot=annot))
}
data=load_inputs(path_to_counts,path_to_annotation) # this will take awhile


loading data...



In [31]:
head(data$cts)

,7316-272,7316-447,7316-898,7316-1702,7316-746,7316-2578,7316-1944,7316-2581,7316-2158,7316-2857,⋯,7316-1080,7316-346,7316-3139,7316-2860,7316-117,7316-2666,7316-725,7316-3319,7316-2723,7316-2758
ENSG00000000003.15_TSPAN6,2470,1385,1269,159,420,3294,1758,3163,1131,1004,⋯,712,1764,843,3670,2428,1092,683,10323,623,5632
ENSG00000000005.6_TNMD,43,9,0,1,1,11,12,1,3,3,⋯,0,7,1,4,6,0,8,21,3,15
ENSG00000000419.14_DPM1,1111,634,738,939,878,1131,586,939,664,1486,⋯,1206,571,787,1194,753,607,740,737,669,875
ENSG00000000457.14_SCYL3,993,547,650,364,1229,911,476,719,666,783,⋯,704,558,663,1101,771,865,635,983,674,287
ENSG00000000460.17_C1orf112,450,437,444,231,594,291,174,294,237,459,⋯,215,147,428,225,210,174,255,393,416,91
ENSG00000000938.13_FGR,284,113,1169,749,245,303,1559,234,867,60,⋯,289,382,486,319,228,289,338,354,376,156


In [32]:
head(data$annot)

,sex,patient_id,tumor_history,age_at_diagnosis,cohort,extent_of_tumor_resection,cancer_type,cancer_subclass,ecDNA_sequences_detected,amplicon_class,in_unique_tumor_set,in_unique_patient_set
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
7316-272,Male,PT_00G007DM,Diagnosis,-1.31769052,PBTA-X00,Gross/Near total resection,ETMR,,1,ecDNA,True,True
7316-447,Male,PT_01MZ62KG,Progressive,-1.28217620,PBTA-X00,Partial resection,ETMR,,1,ecDNA,True,True
7316-898,Male,PT_02J5CWN5,Progressive,0.09335387,PBTA-X00,Partial resection,LGG,MAPK,0,no_amp,True,True
7316-1702,,PT_02MVZZSW,Diagnosis,0.50220160,PBTA-X00,Partial resection,MNG,,0,no_amp,True,True
7316-746,Male,PT_04YTK82N,Diagnosis,-1.05653037,PBTA-X00,Gross/Near total resection,MBL,G3,0,no_amp,True,True
7316-2578,Female,PT_06ERX8Z7,Diagnosis,0.05567405,PBTA-X00,Gross/Near total resection,SEGA,,0,no_amp,True,True


In [39]:
dds <- DESeqDataSetFromMatrix(countData = data$cts,
                              colData = data$annot,
                              design = ~ amplicon_class)
dds <- DESeq(dds)

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 9481 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [ ]:
ddsmf <- DESeqDataSetFromMatrix(countData = data$cts,
                              colData = data$annot,
                              design = ~ tumor_history + age_at_diagnosis + extent_of_tumor_resection + cancer_type + amplicon_class)
ddsmf <- DESeq(ddsmf)

In [ ]:
results(dds,contrast=list(c("ecDNA","ecDNA"),c("chromosomal","no_amp")))

In [29]:
sd

new("standardGeneric", .Data = function (x, na.rm = FALSE) 
standardGeneric("sd"), generic = "sd", package = "BiocGenerics", 
    group = list(), valueClass = character(0), signature = "x", 
    default = new("derivedDefaultMethod", .Data = function (x, 
        na.rm = FALSE) 
    sqrt(var(if (is.vector(x) || is.factor(x)) x else as.double(x), 
        na.rm = na.rm)), target = new("signature", .Data = "ANY", 
        names = "x", package = "methods"), defined = new("signature", 
        .Data = "ANY", names = "x", package = "methods"), generic = "sd"), 
    skeleton = (new("derivedDefaultMethod", .Data = function (x, 
        na.rm = FALSE) 
    sqrt(var(if (is.vector(x) || is.factor(x)) x else as.double(x), 
        na.rm = na.rm)), target = new("signature", .Data = "ANY", 
        names = "x", package = "methods"), defined = new("signature", 
        .Data = "ANY", names = "x", package = "methods"), generic = "sd"))(x, 
        na.rm))
<bytecode: 0x7fbb5dba20e0>
<environment: 0x7fbb5db9bf20>
attr(,"generic")
[1] "sd"
attr(,"generic")attr(,"package")
[1] "BiocGenerics"
attr(,"package")
[1] "BiocGenerics"
attr(,"group")
list()
attr(,"valueClass")
character(0)
attr(,"signature")
[1] "x"
attr(,"default")
Method Definition (Class "derivedDefaultMethod"):

function (x, na.rm = FALSE) 
sqrt(var(if (is.vector(x) || is.factor(x)) x else as.double(x), 
    na.rm = na.rm))
<bytecode: 0x7fbb5db99698>
<environment: namespace:stats>

Signatures:
        x    
target  "ANY"
defined "ANY"
attr(,"skeleton")
(new("derivedDefaultMethod", .Data = function (x, na.rm = FALSE) 
sqrt(var(if (is.vector(x) || is.factor(x)) x else as.double(x), 
    na.rm = na.rm)), target = new("signature", .Data = "ANY", 
    names = "x", package = "methods"), defined = new("signature", 
    .Data = "ANY", names = "x", package = "methods"), generic = "sd"))(x, 
    na.rm)
attr(,"class")
[1] "standardGeneric"
attr(,"class")attr(,"package")
[1] "methods"